In [1]:
from models import metabFoundation
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
output_dir = '/home/jonaheaton/output/RCC-3121'

# Create a new model

model = metabFoundation(
        num_attention_heads = 2,
        num_hidden_layers = 2,
        hidden_size = 16,
        input_size= 2736
)

In [3]:
local_path = f'{output_dir}/pretrain_encoder_state_dict.pth'
encoder_state_dict = torch.load(local_path)

In [4]:
model.load_state_dict(encoder_state_dict)

<All keys matched successfully>

In [5]:
model.get_info()

{'training': True,
 'input_size': 2736,
 'hidden_size': 1,
 'latent_size': 1,
 'num_hidden_layers': 2,
 'num_attention_heads': 2,
 'dropout_rate': 0,
 'activation': 'leakyrelu',
 'use_batch_norm': False,
 'act_on_output_layer': False,
 'goal': 'encode',
 'kind': 'metabFoundation',
 'name': 'metabFoundation',
 'file_id': 'metabFoundation',
 'max_seq_len': 2736,
 'num_encoder_heads': 2,
 'num_decoder_heads': 2,
 'num_encoder_layers': 2,
 'num_decoder_layers': 2,
 'embed_dim': 16,
 'inverse_embed_dim': 16,
 'default_hidden_fraction': 0.2}

In [6]:
model.save_info(output_dir)

In [7]:
X = torch.rand(3, 2736)

In [8]:
z = model.transform(X)

In [9]:
z.shape

torch.Size([3, 2736])

In [10]:

x_hidden = torch.rand_like(X) < 0.2
x_seq, x_pos_ids, x_mask, x_pad = model.metab_to_seq.transform(X,x_hidden)

In [11]:
z, z_enc, z_pos_ids, z_mask, z_pad = model.transform(X,as_seq=True)

In [12]:
z_recon = model.generate(z_enc, x_pos_ids=z_pos_ids, as_seq=True)

In [15]:
x_seq[x_mask]

tensor([0.4426, 0.8706, 0.1391,  ..., 0.6386, 0.9767, 0.0561])

In [16]:
z_recon[x_mask]

tensor([0.5081, 0.8818, 0.3544,  ..., 0.5787, 0.5467, 0.3002],
       grad_fn=<IndexBackward0>)

In [14]:
F.mse_loss(x_seq[x_mask], z_recon[x_mask])* X.size(0)

tensor(0.0931, grad_fn=<MulBackward0>)

In [ ]:
# x_seq, x_pos_ids, x_mask, x_pad = model.metab_to_seq.transform(X)
z, z_enc, z_pos_ids, z_mask, z_pad = model.transform(X,as_seq=True)
z_recon = model.generate(z_enc, x_pos_ids=z_pos_ids, as_seq=True)
# compute only the masked value reconstruction
recon_loss += F.mse_loss(x_seq(x_mask), z_recon(x_mask))* X.size(0)